In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from ISLP import load_data
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix

Question 1

a.

In [3]:
b0 = -4.2
b1 = 0.04
b2 = 0.8
x1 = 60
x2 = 2
exp = b0+b1*x1+b2*x2
p = pow(math.e, exp)/(1+pow(math.e, exp))
p

0.45016600268752205

The probability that this applicant will be admitted to the bootcamp is 0.45.

b.

In [4]:
p = 0.90
exp = math.log(p/(1-p))
x1 = (exp - b0 - b2*x2)/b1
x1


119.9306144334055

The applicant would need about 120 practice hours to have a 90% chance of being admitted.

c. Bo = -4.2 represents the probability of being admitted to the bootcamp if the applicant had zero practice hours and 0 years of prior programming experience. 

In [5]:
x1 = 0
x2 = 0
exp = b0+b1*x1+b2*x2
p = pow(math.e, exp)/(1+pow(math.e, exp))
p

0.014774031693273059

This would give us a probability of 0.015 / 1.5%.

Question 2

In [6]:
OJ = load_data('OJ')
Train = OJ[OJ['StoreID'].isin([1,2,3,4])]
Test = OJ[OJ['StoreID'] == 7]

a.

In [7]:
loyalty_stats = Train['LoyalCH'].agg(['mean','std','median','min','max'])
print(loyalty_stats)
specialCH_stats = Train['SpecialCH'].agg(['mean','std','median','min','max'])
print(specialCH_stats)
priceDiff_stats = Train['PriceDiff'].agg(['mean','std','median','min','max'])
print(priceDiff_stats)

mean      0.519988
std       0.313757
median    0.500000
min       0.000011
max       0.999947
Name: LoyalCH, dtype: float64
mean      0.046218
std       0.210105
median    0.000000
min       0.000000
max       1.000000
Name: SpecialCH, dtype: float64
mean      0.114524
std       0.255935
median    0.200000
min      -0.670000
max       0.440000
Name: PriceDiff, dtype: float64


b.

In [8]:
vars = ['LoyalCH','SpecialCH', 'PriceDiff']
trainSubset = Train[vars]
cor_matrix = trainSubset.corr()
cor_matrix

,LoyalCH,SpecialCH,PriceDiff
LoyalCH,1.000000,0.014230,0.008349
SpecialCH,0.014230,1.000000,-0.025542
PriceDiff,0.008349,-0.025542,1.000000


c.

In [11]:
Train['Purchase_binary'] = (Train['Purchase'] == 'CH').astype(int)
X = Train[['LoyalCH', 'SpecialCH', 'PriceDiff']]
X = sm.add_constant(X)
y = Train['Purchase_binary']
model = sm.Logit(y,X)
result = model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.414155
         Iterations 7


C:\Users\kate\AppData\Local\Temp\ipykernel_1212\1076967648.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Train['Purchase_binary'] = (Train['Purchase'] == 'CH').astype(int)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        Purchase_binary   No. Observations:                  714
Model:                          Logit   Df Residuals:                      710
Method:                           MLE   Df Model:                            3
Date:                Wed, 15 Oct 2025   Pseudo R-squ.:                  0.4009
Time:                        10:33:32   Log-Likelihood:                -295.71
converged:                       True   LL-Null:                       -493.55
Covariance Type:            nonrobust   LLR p-value:                 1.902e-85
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.4069      0.267    -12.756      0.000      -3.930      -2.883
LoyalCH        6.3847      0.456     14.009      0.000       5.491       7.278
SpecialCH     -0.1841      0.447     -0.412      0.680      -1.060       0.691
PriceDiff      2.5697      0.426      6.038      0.000       1.736       3.404
==============================================================================
"""

According to this logistic regression model, both the loyalty for CH and price difference are statistically significant. I can tell they are significant by looking at the p values and seeing that both of them are 0. On the oher hand, the existence of a special on CH is not statistically significant in this model since the p value is 0.680.

d.

In [14]:
y_pred_prob = result.predict(X)
y_pred = (y_pred_prob >= 0.5).astype(int)
cm = confusion_matrix(y, y_pred)
cm


array([[269,  66],
       [ 62, 317]])

In [16]:
TN, FP, FN, TP = cm.ravel()

accuracy = (TP + TN) / (TP + TN + FP + FN)
false_positive_rate = FP / (FP + TN)
false_negative_rate = FN / (FN + TP)
print(f"Accuracy {accuracy}")
print(f"False positive rate {false_positive_rate}")
print(f"False negative rate {false_negative_rate}")

Accuracy 0.8207282913165266
False positive rate 0.19701492537313434
False negative rate 0.16358839050131926


e.

In [17]:
X_train2 = Train[['LoyalCH', 'PriceDiff']]
X_train2 = sm.add_constant(X_train2)
y_train = Train["Purchase_binary"]

model2 = sm.Logit(y_train, X_train2)
result2 = model2.fit()
result2.summary()

Optimization terminated successfully.
         Current function value: 0.414274
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        Purchase_binary   No. Observations:                  714
Model:                          Logit   Df Residuals:                      711
Method:                           MLE   Df Model:                            2
Date:                Wed, 15 Oct 2025   Pseudo R-squ.:                  0.4007
Time:                        10:51:25   Log-Likelihood:                -295.79
converged:                       True   LL-Null:                       -493.55
Covariance Type:            nonrobust   LLR p-value:                 1.301e-86
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.4159      0.266    -12.820      0.000      -3.938      -2.894
LoyalCH        6.3811      0.456     14.008      0.000       5.488       7.274
PriceDiff      2.5729      0.426      6.043      0.000       1.739       3.407
==============================================================================
"""

In [22]:
Test['Purchase_binary'] = (Test['Purchase'] == 'CH').astype(int)
X_test2 = Test[['LoyalCH', 'PriceDiff']]
X_test2 = sm.add_constant(X_test2)
y_test = Test['Purchase_binary']

y_pred_prob_test = result2.predict(X_test2)
y_pred_test = (y_pred_prob_test >= 0.5).astype(int)

cm_test = confusion_matrix(y_test, y_pred_test)
cm_test

C:\Users\kate\AppData\Local\Temp\ipykernel_1212\2553722809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test['Purchase_binary'] = (Test['Purchase'] == 'CH').astype(int)


array([[ 60,  22],
       [ 34, 240]])

In [23]:
TN, FP, FN, TP = cm_test.ravel()

accuracy = (TP + TN) / (TP + TN + FP + FN)
false_positive_rate = FP / (FP + TN)
false_negative_rate = FN / (FN + TP)
print(f"Accuracy {accuracy}")
print(f"False positive rate {false_positive_rate}")
print(f"False negative rate {false_negative_rate}")


Accuracy 0.8426966292134831
False positive rate 0.2682926829268293
False negative rate 0.12408759124087591
